# Combine datasets together

In [1]:
# Import libraries
import os #operating system
import glob # for reading multiple files
from glob import glob
import pandas as pd #pandas for dataframe management
import matplotlib.pyplot as plt #matplotlib for plotting
import matplotlib.dates as mdates # alias for date formatting
import numpy as np # for generating synthetic data

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [2]:
# Set some variables
dataPath = '../data' # set data path

In [3]:
# Define a function to read the data
def read_data(file):
# Read in excel 
    df = (pd.read_excel(
        file,
        skiprows=range(1,9),
        usecols='B:C',
        header=0,
    ))

    # remove last rows
    df.drop(
        df.tail(2).index,
        inplace=True
    )

    # fix index and naming
    df.columns = ['date', 'demand']
    df['date'] = pd.to_datetime(df['date']) # convert column to datetime
    df.set_index('date', inplace=True)

    return df

# Read all the data

In [4]:
# Create a list of files to combine
PATH = dataPath
EXT = "*.xls"
all_files = [file
            for path, subdir, files in os.walk(PATH)
            for file in glob(os.path.join(path, EXT))]

# Assemble files into a final dataframe
df = pd.DataFrame()
for file in all_files:
    tmp = read_data(file)
    dfs = [df, tmp]
    df = pd.concat(dfs)

# Data processing

In [78]:
df.sort_index(inplace=True)
df['demand'] = df['demand'].apply(pd.to_numeric, errors='coerce')

ts_daily = df.resample('D').mean()
days = ts_daily.index.strftime("%Y-%m-%d")

ts_monthly = df.resample('M').mean()
months = ts_monthly.index.strftime("%Y-%m")

## Daily

In [79]:
def create_date_labels(df):
    """ Function to create day labels
    """
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['weekday'] = df.index.weekday
    df['month_name'] = df.index.month_name()
    df['day_name'] = df.index.day_name()

    return df

In [80]:
ts_daily = create_date_labels(ts_daily)

### Filter by weekends and holidays

In [81]:
ts_daily.loc[ts_daily['weekday'] < 5] # Weekday
ts_daily.loc[ts_daily['weekday'] >= 5] # Weekend


,demand,year,month,weekday,month_name,day_name
date,,,,,,
2019-01-05,47.747368,2019,1,5,January,Saturday
2019-01-06,46.443789,2019,1,6,January,Sunday
2019-01-12,33.711158,2019,1,5,January,Saturday
2019-01-13,39.518351,2019,1,6,January,Sunday
2019-01-19,38.954227,2019,1,5,January,Saturday
...,...,...,...,...,...,...
2019-12-15,33.788041,2019,12,6,December,Sunday
2019-12-21,31.858144,2019,12,5,December,Saturday
2019-12-22,31.680000,2019,12,6,December,Sunday


In [83]:
from datetime import date
import holidays
# from pandas.tseries.holiday 

In [84]:
brazil_holidays = holidays.Brazil()
for ptr in holidays.Brazil(years = 2019).items():
    print(str(ptr[0]))

2019-01-01
2019-04-21
2019-05-01
2019-09-07
2019-10-12
2019-11-02
2019-11-15
2019-12-25
2019-04-19
2019-06-20
2019-03-06
2019-03-05


In [88]:
ts_daily.head()

,demand,year,month,weekday,month_name,day_name
date,,,,,,
2019-01-01,36.833684,2019,1,1,January,Tuesday
2019-01-02,42.563368,2019,1,2,January,Wednesday
2019-01-03,49.657263,2019,1,3,January,Thursday
2019-01-04,69.544421,2019,1,4,January,Friday
2019-01-05,47.747368,2019,1,5,January,Saturday


In [89]:
ts_daily.reset_index(inplace=True)

In [93]:
brazil_holidays.get('2019-03-05')

'Carnaval'

In [97]:
ts_daily.loc[ts_daily['date']]

SyntaxError: invalid syntax (<ipython-input-97-a9e1d8f2056f>, line 1)

In [99]:
ts_daily['holiday'] = brazil_holidays(ts_daily['date'])

TypeError: 'Brazil' object is not callable

In [92]:
ts_daily['date']

0     2019-01-01
1     2019-01-02
2     2019-01-03
3     2019-01-04
4     2019-01-05
         ...    
360   2019-12-27
361   2019-12-28
362   2019-12-29
363   2019-12-30
364   2019-12-31
Name: date, Length: 365, dtype: datetime64[ns]

In [66]:
str(ts_daily.index.values[0]) in brazil_holidays

True